# Setup: Test & Explore

Start here!
This file is used to test and explore the functions and get a better understanding of **prompt engineering, vector databases & the whole Retrieval-augmented generation (RAG) architecture**.

In [1]:
# important for changing code in the modules without restarting the Kernel
%load_ext autoreload
%autoreload 2

# import local functions
from TalkToOllama import LLMOllama # --> TalkToOllama.py
from TalkToDatabase import VectorDB # --> TalkToDatabase.py
from ImportData import DataHandler # --> ImportData.py

# Logging
import logging
log = logging.getLogger("applogger")
logging.basicConfig(level=logging.INFO)

## Connection & Instances

In [2]:
vectordb = VectorDB(
  collection_name="catcenter_vectors",
  database_path="chromadb/"
)

LLM = LLMOllama(
  database=vectordb,
  model="llama3.1"
)

datahandler = DataHandler(
  database=vectordb
)

INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


## Import Data to vector database

In [3]:
# Scrape data from web-docs
datahandler.scrape_apidocs_catcenter()

# Import from Specfication file
datahandler.import_apispecs_from_json()

INFO:applogger:['Cisco Catalyst Center Platform APIs and Integrations Overview. - Cisco Catalyst Center 2.3.7 - Cisco DevNet                          Cisco Catalyst Center Platform APIs and Integrations Overview. This page explains Catalyst Center integration options: Intent APIs provide REST access to the Catalyst Center platform. Integration APIs enable IT Service Management workflows. Events and Notifications WebHooks enables notification on key events. The Multivendor API enables direct support for non-Cisco devi', 'ces. Cisco Catalyst Center Platform Overview Cisco Catalyst Center is at the heart of Cisco’s intent-based network architecture. Catalyst Center supports the expression of business intent for network use cases, such as base automation capabilities in the enterprise network. The Analytics and Assurance features of Catalyst Center provide end-to-end visibility into the network with full context through data and insights. Cisco customers and partners can use the Catalyst C

## Understand Tokenization & Embeddings

In [4]:
from sentence_transformers import SentenceTransformer

# all-MiniLM-L6-v2 generates vectors with 384 dimensions
model = SentenceTransformer("all-MiniLM-L6-v2") 

# Encoding text
text = "Cisco Catalyst Center is at the heart of Cisco’s intent-based network architecture. Catalyst Center supports the expression of business intent for network use cases, such as base automation capabilities in the enterprise network. The Analytics and Assurance features of Catalyst Center provide end-to-end visibility into the network with full context through data and insights."
embedding = model.encode(text,output_value=None)

print("Text:", text)
print("Token IDs:", embedding["input_ids"])
print("Embedding:", embedding["sentence_embedding"])

/Users/flpachin/Library/Caches/pypoetry/virtualenvs/understanding-rag-catalyst-center-M5QtixxY-py3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Batches: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00

Text: Cisco Catalyst Center is at the heart of Cisco’s intent-based network architecture. Catalyst Center supports the expression of business intent for network use cases, such as base automation capabilities in the enterprise network. The Analytics and Assurance features of Catalyst Center provide end-to-end visibility into the network with full context through data and insights.
Token IDs: tensor([  101, 26408, 16771,  2415,  2003,  2012,  1996,  2540,  1997, 26408,
         1521,  1055,  7848,  1011,  2241,  2897,  4294,  1012, 16771,  2415,
         6753,  1996,  3670,  1997,  2449,  7848,  2005,  2897,  2224,  3572,
         1010,  2107,  2004,  2918, 19309,  9859,  1999,  1996,  6960,  2897,
         1012,  1996, 25095,  1998, 16375,  2838,  1997, 16771,  2415,  3073,
         2203,  1011,  2000,  1011,  2203, 16476,  2046,  1996,  2897,  2007,
         2440,  6123,  2083,  2951,  1998, 20062,  1012,   102],
       device='mps:0')
Embedding: tensor([-1.5361e-02,  1.0563e-02, -5.6

## Explore Vectore Database

In [5]:
# The total number of embeddings added to the database:
vectordb.collection.count()

1019

In [6]:
# similarity search with the defined keyword
db_query = ["eox status"]
number_of_results = 3

vectordb_query = vectordb.collection.query(
        query_texts = db_query,
        n_results = number_of_results,
        include = ["embeddings", "metadatas", "documents","distances"]
)
vectordb_query

{'ids': [['getEoXSummary_0',
   'getEoXStatusForAllDevices_0',
   'getEoXDetailsPerDevice_0']],
 'embeddings': [array([[-0.04588801,  0.08671391,  0.0407329 , ...,  0.06532472,
           0.02179648,  0.04572295],
         [-0.04651923,  0.10227174,  0.0326543 , ...,  0.05825355,
           0.03559346,  0.04987536],
         [-0.06667568,  0.10506882,  0.03106867, ...,  0.0642276 ,
           0.04445031,  0.02198924]])],
 'documents': [['The provided REST API query path "/dna/intent/api/v1/eox-status/summary" with the operation "get" is used to retrieve the End of Sale (EoX) summary for all devices in the network. This query enables users to access a consolidated overview of the EoX status of all devices present in their network.  From a user perspective, this query is valuable for network administrators, operators, and IT personnel responsible for managing and maintaining network infrastructure. By making this query, users can easily obtain critical information about the EoX status of

## Talk to LLM

Prompting guide: https://llama.meta.com/docs/how-to-guides/prompting/ 

In [7]:
LLM.ask_llm("What is Cisco?")

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


{'model': 'llama3.1',
 'created_at': '2025-02-03T13:22:38.642957Z',
 'message': {'role': 'assistant',
  'content': 'Cisco is a well-known American multinational technology conglomerate that specializes in designing, manufacturing, and selling networking hardware, telecommunications equipment, and other high-technology services.\n\nThe company was founded in 1984 by Leonard Bosack and Sandy Lerner, and it\'s named after the San Francisco Bay (Ciscos being a colloquialism for "San Francis Cos" or just "Cisco"). Over time, Cisco has grown to become one of the largest and most influential technology companies in the world.\n\nSome of the key products and services offered by Cisco include:\n\n1. Routers: Devices that help direct traffic on computer networks.\n2. Switches: Hardware components that connect devices within a network.\n3. Network security systems: Solutions for protecting against cyber threats and vulnerabilities.\n4. Unified Communications (UC) solutions: Tools for integrating 

In [12]:
ask = LLM.ask_llm_rag(
    query_string="eox status of all devices",
    n_results=5
)

INFO:applogger:=== USER PROMPT === 
 User question: 'eox status of all devices'
    Context delimited with XML tags: <context>[['The provided REST API query path "/dna/intent/api/v1/eox-status/summary" with the operation "get" is used to retrieve the End of Sale (EoX) summary for all devices in the network. This query enables users to access a consolidated overview of the EoX status of all devices present in their network.  From a user perspective, this query is valuable for network administrators, operators, and IT personnel responsible for managing and maintaining network infrastructure. By making this query, users can easily obtain critical information about the EoX status of all devices in their network. This information can include details such as upcoming end-of-life dates for hardware and software components, recommended upgrades or replacements, and any relevant notices issued by Cisco regarding the devices.  By utilizing this query, users can stay proactive in planning for nec

In [11]:
# Makes the Markdown-Output nicer
from IPython.display import Markdown, display
display(Markdown(ask)) #LLM Output from the ask variable

Based on the provided context, I will guide you through the steps to retrieve the End of Sale (EoX) status for all devices in your network using the Cisco Catalyst Center REST API.

**Step 1: Obtain an Access Token**

Before making any API calls, we need to obtain an access token. This is a one-time process unless the token expires.

```python
import requests

# Basic authentication for obtaining the access token
auth_url = "/dna/system/api/v1/auth/token"
username = "your_username"
password = "your_password"

response = requests.post(auth_url, auth=("your_username", "your_password"), headers={"Content-Type": "application/json"})
access_token = response.json()["Access-Token"]

# Store the access token for future API calls
headers = {"X-Auth-Token": access_token}
```

**Step 2: Retrieve EoX Status Summary for All Devices**

We will use the REST API query path `/dna/intent/api/v1/eox-status/summary` with the operation `get`.

```python
# EoX status summary for all devices
eox_summary_url = "/dna/intent/api/v1/eox-status/summary"
response = requests.get(eox_summary_url, headers=headers)
print(response.json())
```

**Step 3: Retrieve EoX Status for All Devices**

We will use the REST API query path `/dna/intent/api/v1/eox-status/device` with the operation `get`.

```python
# EoX status for all devices
eox_device_url = "/dna/intent/api/v1/eox-status/device"
response = requests.get(eox_device_url, headers=headers)
print(response.json())
```

**Step 4: Retrieve EoX Status for a Specific Device**

We will use the REST API query path `/dna/intent/api/v1/eox-status/device/{deviceId}` with the operation `get`. Replace `{deviceId}` with the actual device instance UUID.

```python
# EoX status for a specific device
device_id = "your_device_id"
eox_device_specific_url = f"/dna/intent/api/v1/eox-status/device/{device_id}"
response = requests.get(eox_device_specific_url, headers=headers)
print(response.json())
```

**Code Snippet**

```python
import requests

# Basic authentication for obtaining the access token
auth_url = "/dna/system/api/v1/auth/token"
username = "your_username"
password = "your_password"

response = requests.post(auth_url, auth=("your_username", "your_password"), headers={"Content-Type": "application/json"})
access_token = response.json()["Access-Token"]

# Store the access token for future API calls
headers = {"X-Auth-Token": access_token}

# EoX status summary for all devices
eox_summary_url = "/dna/intent/api/v1/eox-status/summary"
response = requests.get(eox_summary_url, headers=headers)
print(response.json())

# EoX status for all devices
eox_device_url = "/dna/intent/api/v1/eox-status/device"
response = requests.get(eox_device_url, headers=headers)
print(response.json())

# Replace {deviceId} with the actual device instance UUID
device_id = "your_device_id"
eox_device_specific_url = f"/dna/intent/api/v1/eox-status/device/{device_id}"
response = requests.get(eox_device_specific_url, headers=headers)
print(response.json())
```

**Required Steps and Documentation**

* Make sure to replace `your_username` and `your_password` with your actual credentials.
* Replace `{deviceId}` in the last API call with the actual device instance UUID.
* The code snippet assumes you have the `requests` library installed. If not, install it using pip: `pip install requests`.

**Note**: This is a basic example to get you started. You may need to modify the code based on your specific requirements and environment.

I hope this helps! If you have any further questions or need additional assistance, feel free to ask.

The query 'eox status of all devices' took **38.85 seconds** to execute.